In [0]:
with cte_data (bank_str, bank) as
(
select value
     , transform(split(value, ''), (t,i) -> struct(t::tinyint as n, length(value) - i as ord, i + 1 as pos))
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_03_input.txt', format => 'text', wholeText => false) as t (value)
/*from values ('987654321111111')
            , ('811111111111119')
            , ('234234234234278')
            , ('818181911112111') as t (value)*/
)
  ,  cte_tmp (n1, n2, p2) as
(
select array_max(split(substring(bank_str from 1 for length(bank_str) - 1), '')) as n1
     , array_max(split(substring(bank_str from instr(bank_str, n1) + 1)   , '')) as n2
     , aggregate( sequence(11,0,-1)
                , array(struct(0y as n, 0 as ord, 0 as pos))
                , (acc, x) -> array_append( acc
                                          , sort_array(slice(bank, element_at(acc, -1).pos + 1, array_size(bank) - x - element_at(acc, -1).pos), false)[0]
                                          )
                , acc -> aggregate(sequence(11,0,-1), 0l, (agg, x) -> agg + (element_at(acc, 13 - x).n * power(10, x))::bigint)
                )
  from cte_data
)
select sum(n1::int * 10 + n2::int) as part1
     , sum(p2)                     as part2
  from cte_tmp;

